In [2]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [3]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [4]:
pair = "EUR_USD"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [5]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2021-03-31 01:00:00+00:00,5781,1.17224,1.17241,1.17042,1.17077,1.17219,1.17234,1.17035,1.17070,1.17230,1.17248,1.17048,1.17084
1,2021-03-31 05:00:00+00:00,16528,1.17078,1.17481,1.17048,1.17464,1.17072,1.17475,1.17042,1.17457,1.17085,1.17487,1.17053,1.17470
2,2021-03-31 09:00:00+00:00,16080,1.17462,1.17463,1.17212,1.17256,1.17455,1.17456,1.17205,1.17249,1.17469,1.17470,1.17218,1.17262
3,2021-03-31 13:00:00+00:00,23629,1.17257,1.17600,1.17211,1.17500,1.17251,1.17594,1.17206,1.17494,1.17263,1.17606,1.17216,1.17505
4,2021-03-31 17:00:00+00:00,10216,1.17498,1.17554,1.17196,1.17308,1.17492,1.17548,1.17189,1.17299,1.17504,1.17560,1.17202,1.17317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,2023-10-20 01:00:00+00:00,8879,1.05760,1.05800,1.05684,1.05754,1.05752,1.05793,1.05677,1.05746,1.05768,1.05808,1.05691,1.05761
3997,2023-10-20 05:00:00+00:00,16930,1.05750,1.05953,1.05650,1.05877,1.05743,1.05946,1.05643,1.05869,1.05758,1.05960,1.05658,1.05885
3998,2023-10-20 09:00:00+00:00,17538,1.05875,1.05990,1.05652,1.05838,1.05868,1.05983,1.05644,1.05831,1.05882,1.05997,1.05660,1.05846
3999,2023-10-20 13:00:00+00:00,24510,1.05838,1.06009,1.05722,1.05896,1.05831,1.06002,1.05714,1.05888,1.05845,1.06016,1.05730,1.05903


In [6]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [7]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2021-03-31 01:00:00+00:00,1.17224,1.17241,1.17042,1.17077
1,2021-03-31 05:00:00+00:00,1.17078,1.17481,1.17048,1.17464
2,2021-03-31 09:00:00+00:00,1.17462,1.17463,1.17212,1.17256
3,2021-03-31 13:00:00+00:00,1.17257,1.17600,1.17211,1.17500
4,2021-03-31 17:00:00+00:00,1.17498,1.17554,1.17196,1.17308


In [8]:
for ma in MA_LIST:    
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [9]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,MA_50,MA_100,MA_200
0,2021-05-17 05:00:00+00:00,1.21298,1.21500,1.21262,1.21481,1.211896,1.210487,1.209959,1.208530,1.199507
1,2021-05-17 09:00:00+00:00,1.21480,1.21690,1.21346,1.21400,1.212506,1.210586,1.210194,1.208656,1.199723
2,2021-05-17 13:00:00+00:00,1.21398,1.21577,1.21380,1.21560,1.213313,1.210693,1.210512,1.208791,1.199928
3,2021-05-17 17:00:00+00:00,1.21561,1.21615,1.21496,1.21500,1.213955,1.210793,1.210818,1.208914,1.200140
4,2021-05-17 21:00:00+00:00,1.21528,1.21590,1.21508,1.21564,1.214267,1.211243,1.211143,1.209012,1.200343


In [10]:
df_plot = df_ma.iloc[:500]

In [11]:
cp = CandlePlot(df_plot)

In [12]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [13]:
traces

['MA_10', 'MA_20', 'MA_50', 'MA_100', 'MA_200']

In [14]:
cp.show_plot(line_traces=traces)

In [15]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [16]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [17]:
df_an

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2021-05-17 05:00:00+00:00,1.21298,1.21500,1.21262,1.21481,1.209959,1.199507
1,2021-05-17 09:00:00+00:00,1.21480,1.21690,1.21346,1.21400,1.210194,1.199723
2,2021-05-17 13:00:00+00:00,1.21398,1.21577,1.21380,1.21560,1.210512,1.199928
3,2021-05-17 17:00:00+00:00,1.21561,1.21615,1.21496,1.21500,1.210818,1.200140
4,2021-05-17 21:00:00+00:00,1.21528,1.21590,1.21508,1.21564,1.211143,1.200343
...,...,...,...,...,...,...,...
3797,2023-10-20 01:00:00+00:00,1.05760,1.05800,1.05684,1.05754,1.056792,1.061595
3798,2023-10-20 05:00:00+00:00,1.05750,1.05953,1.05650,1.05877,1.056822,1.061492
3799,2023-10-20 09:00:00+00:00,1.05875,1.05990,1.05652,1.05838,1.056867,1.061388
3800,2023-10-20 13:00:00+00:00,1.05838,1.06009,1.05722,1.05896,1.056837,1.061309


In [18]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [19]:
df_an

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV
0,2021-05-17 05:00:00+00:00,1.21298,1.21500,1.21262,1.21481,1.209959,1.199507,0.010452,NaN
1,2021-05-17 09:00:00+00:00,1.21480,1.21690,1.21346,1.21400,1.210194,1.199723,0.010471,0.010452
2,2021-05-17 13:00:00+00:00,1.21398,1.21577,1.21380,1.21560,1.210512,1.199928,0.010585,0.010471
3,2021-05-17 17:00:00+00:00,1.21561,1.21615,1.21496,1.21500,1.210818,1.200140,0.010678,0.010585
4,2021-05-17 21:00:00+00:00,1.21528,1.21590,1.21508,1.21564,1.211143,1.200343,0.010800,0.010678
...,...,...,...,...,...,...,...,...,...
3797,2023-10-20 01:00:00+00:00,1.05760,1.05800,1.05684,1.05754,1.056792,1.061595,-0.004803,-0.004930
3798,2023-10-20 05:00:00+00:00,1.05750,1.05953,1.05650,1.05877,1.056822,1.061492,-0.004670,-0.004803
3799,2023-10-20 09:00:00+00:00,1.05875,1.05990,1.05652,1.05838,1.056867,1.061388,-0.004521,-0.004670
3800,2023-10-20 13:00:00+00:00,1.05838,1.06009,1.05722,1.05896,1.056837,1.061309,-0.004471,-0.004521


In [20]:
def is_trade(row):
    if row.DELTA >= 0  and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0  and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [21]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [22]:
df_an

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
0,2021-05-17 05:00:00+00:00,1.21298,1.21500,1.21262,1.21481,1.209959,1.199507,0.010452,NaN,0
1,2021-05-17 09:00:00+00:00,1.21480,1.21690,1.21346,1.21400,1.210194,1.199723,0.010471,0.010452,0
2,2021-05-17 13:00:00+00:00,1.21398,1.21577,1.21380,1.21560,1.210512,1.199928,0.010585,0.010471,0
3,2021-05-17 17:00:00+00:00,1.21561,1.21615,1.21496,1.21500,1.210818,1.200140,0.010678,0.010585,0
4,2021-05-17 21:00:00+00:00,1.21528,1.21590,1.21508,1.21564,1.211143,1.200343,0.010800,0.010678,0
...,...,...,...,...,...,...,...,...,...,...
3797,2023-10-20 01:00:00+00:00,1.05760,1.05800,1.05684,1.05754,1.056792,1.061595,-0.004803,-0.004930,0
3798,2023-10-20 05:00:00+00:00,1.05750,1.05953,1.05650,1.05877,1.056822,1.061492,-0.004670,-0.004803,0
3799,2023-10-20 09:00:00+00:00,1.05875,1.05990,1.05652,1.05838,1.056867,1.061388,-0.004521,-0.004670,0
3800,2023-10-20 13:00:00+00:00,1.05838,1.06009,1.05722,1.05896,1.056837,1.061309,-0.004471,-0.004521,0


In [23]:
df_an.to_csv("trade.csv")

In [24]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [25]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
135,2021-06-16 17:00:00+00:00,1.21174,1.21175,1.19946,1.19951,1.214937,1.214945,-0.000008,0.000293,-1
350,2021-08-05 13:00:00+00:00,1.18360,1.18514,1.18320,1.18388,1.185008,1.184891,0.000117,-0.000020,1
366,2021-08-10 05:00:00+00:00,1.17383,1.17428,1.17262,1.17322,1.183593,1.183679,-0.000086,0.000038,-1
471,2021-09-02 17:00:00+00:00,1.18649,1.18761,1.18644,1.18750,1.178691,1.178472,0.000219,-0.000023,1
552,2021-09-22 05:00:00+00:00,1.17218,1.17376,1.17170,1.17342,1.177592,1.177662,-0.000070,0.000064,-1


In [26]:
sell_trades = df_an[df_an.TRADE == -1].copy()
sell_trades

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
135,2021-06-16 17:00:00+00:00,1.21174,1.21175,1.19946,1.19951,1.214937,1.214945,-0.000008,0.000293,-1
366,2021-08-10 05:00:00+00:00,1.17383,1.17428,1.17262,1.17322,1.183593,1.183679,-0.000086,0.000038,-1
552,2021-09-22 05:00:00+00:00,1.17218,1.17376,1.17170,1.17342,1.177592,1.177662,-0.000070,0.000064,-1
1099,2022-01-27 10:00:00+00:00,1.11962,1.12010,1.11350,1.11494,1.132379,1.132570,-0.000191,0.000274,-1
1215,2022-02-23 18:00:00+00:00,1.13122,1.13125,1.13012,1.13046,1.134378,1.134519,-0.000141,0.000033,-1
1696,2022-06-15 21:00:00+00:00,1.04444,1.04694,1.04426,1.04528,1.058744,1.058941,-0.000197,0.000307,-1
1984,2022-08-22 21:00:00+00:00,0.99430,0.99501,0.99302,0.99398,1.016639,1.016670,-0.000031,0.000623,-1
2729,2023-02-14 02:00:00+00:00,1.07286,1.07400,1.07255,1.07384,1.077191,1.077214,-0.000023,0.000543,-1
3120,2023-05-16 05:00:00+00:00,1.08830,1.08978,1.08629,1.08914,1.096063,1.096156,-0.000093,0.000303,-1
3466,2023-08-03 21:00:00+00:00,1.09457,1.09556,1.09444,1.09556,1.100982,1.101046,-0.000064,0.000208,-1


In [27]:
cp = CandlePlot(df_an)
cp.show_plot(line_traces=[MA_S, MA_L])

In [28]:
ic.LoadInstruments("../data")

In [29]:
ic.instruments_dict[pair]

{'name': 'EUR_USD', 'ins_type': 'CURRENCY', 'displayName': 'EUR/USD', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.02}

In [30]:
ins_data = ic.instruments_dict[pair]

In [31]:
df_trades.shape

(19, 10)

In [32]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [33]:
df_trades

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE,DIFF
135,2021-06-16 17:00:00+00:00,1.21174,1.21175,1.19946,1.19951,1.214937,1.214945,-0.000008,0.000293,-1,-0.01563
350,2021-08-05 13:00:00+00:00,1.18360,1.18514,1.18320,1.18388,1.185008,1.184891,0.000117,-0.000020,1,-0.01066
366,2021-08-10 05:00:00+00:00,1.17383,1.17428,1.17262,1.17322,1.183593,1.183679,-0.000086,0.000038,-1,0.01428
471,2021-09-02 17:00:00+00:00,1.18649,1.18761,1.18644,1.18750,1.178691,1.178472,0.000219,-0.000023,1,-0.01408
552,2021-09-22 05:00:00+00:00,1.17218,1.17376,1.17170,1.17342,1.177592,1.177662,-0.000070,0.000064,-1,-0.04236
968,2021-12-28 14:00:00+00:00,1.13213,1.13230,1.12896,1.13106,1.130735,1.130675,0.000060,-0.000011,1,-0.01612
1099,2022-01-27 10:00:00+00:00,1.11962,1.12010,1.11350,1.11494,1.132379,1.132570,-0.000191,0.000274,-1,0.02862
1153,2022-02-09 10:00:00+00:00,1.14266,1.14460,1.14234,1.14356,1.133375,1.133249,0.000126,-0.000384,1,-0.01310
1215,2022-02-23 18:00:00+00:00,1.13122,1.13125,1.13012,1.13046,1.134378,1.134519,-0.000141,0.000033,-1,-0.05744
1618,2022-05-29 21:00:00+00:00,1.07375,1.07375,1.07262,1.07302,1.063323,1.063310,0.000013,-0.000417,1,-0.02774


In [34]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE'] 

In [38]:
df_trades

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,GAIN_C
135,2021-06-16 17:00:00+00:00,1.21174,1.21175,1.19946,1.19951,1.214937,1.214945,-0.000008,0.000293,-1,-0.01563,156.3,156.3
350,2021-08-05 13:00:00+00:00,1.18360,1.18514,1.18320,1.18388,1.185008,1.184891,0.000117,-0.000020,1,-0.01066,-106.6,49.7
366,2021-08-10 05:00:00+00:00,1.17383,1.17428,1.17262,1.17322,1.183593,1.183679,-0.000086,0.000038,-1,0.01428,-142.8,-93.1
471,2021-09-02 17:00:00+00:00,1.18649,1.18761,1.18644,1.18750,1.178691,1.178472,0.000219,-0.000023,1,-0.01408,-140.8,-233.9
552,2021-09-22 05:00:00+00:00,1.17218,1.17376,1.17170,1.17342,1.177592,1.177662,-0.000070,0.000064,-1,-0.04236,423.6,189.7
968,2021-12-28 14:00:00+00:00,1.13213,1.13230,1.12896,1.13106,1.130735,1.130675,0.000060,-0.000011,1,-0.01612,-161.2,28.5
1099,2022-01-27 10:00:00+00:00,1.11962,1.12010,1.11350,1.11494,1.132379,1.132570,-0.000191,0.000274,-1,0.02862,-286.2,-257.7
1153,2022-02-09 10:00:00+00:00,1.14266,1.14460,1.14234,1.14356,1.133375,1.133249,0.000126,-0.000384,1,-0.01310,-131.0,-388.7
1215,2022-02-23 18:00:00+00:00,1.13122,1.13125,1.13012,1.13046,1.134378,1.134519,-0.000141,0.000033,-1,-0.05744,574.4,185.7
1618,2022-05-29 21:00:00+00:00,1.07375,1.07375,1.07262,1.07302,1.063323,1.063310,0.000013,-0.000417,1,-0.02774,-277.4,-91.7


In [35]:
df_trades.GAIN.sum()

270.29999999999893

In [36]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [37]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])